# We are used to pandas, aren't we?

(please upvote if it helps, it motivates me;)

The official API to access the data uses original data-class.
I am not familiar with that data format, so I just made a reading api for pandas dataframe.

This is WIP notebook, as just one data is treated.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
def floorname2num(floorname):
    if floorname == "F11" or floorname == "11F" or floorname == "L11" or floorname == "11L":
        return 10
    if floorname == "F10" or floorname == "10F" or floorname == "L10" or floorname == "10L":
        return 9
    if floorname == "F9" or floorname == "9F" or floorname == "L9" or floorname == "9L":
        return 8
    if floorname == "F8" or floorname == "8F" or floorname == "L8" or floorname == "8L":
        return 7
    if floorname == "F7" or floorname == "7F" or floorname == "L7" or floorname == "7L":
        return 6
    if floorname == "F6" or floorname == "6F" or floorname == "L6" or floorname == "6L":
        return 5
    if floorname == "F5" or floorname == "5F" or floorname == "L5" or floorname == "5L":
        return 4
    if floorname == "F4" or floorname == "4F" or floorname == "L4" or floorname == "4L":
        return 3
    if floorname == "F3" or floorname == "3F" or floorname == "L3" or floorname == "3L":
        return 2
    if floorname == "F2" or floorname == "2F" or floorname == "L2" or floorname == "2L":
        return 1
    if floorname == "F1" or floorname == "1F" or floorname == "L1" or floorname == "1L":
        return 0
    if floorname == "B1" or floorname == "1B" or floorname == "BF" or floorname == "B":
        return -1
    if floorname == "B2" or floorname == "2B":
        return -2
    if floorname == "B3" or floorname == "3B":
        return -3
        
    return None
    # ignoring following floor names
    # M at 玫瑰坊 and ../input/indoor-location-navigation/train/5cdac620e403deddaf467ff9 (亚新生活广场)
    # ../input/indoor-location-navigation/train/5cd969bd39e2fc0b4afe727d (绿地缤纷城(徐汇店))
    # BM at ../input/indoor-location-navigation/train/5d2709dd03f801723c32cfb6 (印象城(西溪店))
    # LG1, LG2, P1, P2 at ../input/indoor-location-navigation/train/5cd969ba39e2fc0b4afe6fae (兴业太古汇)
    # LG1, LG2 at ../input/indoor-location-navigation/train/5cd56bd9e2acfd2d33b662df (晶品)
    #          and ../input/indoor-location-navigation/train/5cd56bd6e2acfd2d33b65dca (尚嘉中心)
    # G, LM at ../input/indoor-location-navigation/train/5cd56bc1e2acfd2d33b6404d (世博源) ... this also has B and L1 


In [ ]:
# copy and modified from https://github.com/location-competition/indoor-location-competition-20/blob/master/io_f.py

def read_one_data_file(data_filename, return_metadata=True):
    
    with open(data_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        
    metadata = {}
    data_table = []

    for line_data in lines:
        
        line_data = line_data.strip()
        if not line_data :
            continue
        if line_data[0] == '#':
            # read metadata that looks useful
            line_data = line_data.split('\t')
            if "SiteID" in line_data[1] :
                metadata["site_id"] = line_data[1].split(":")[-1]
                continue
                
            if "Brand" in line_data[1] :
                metadata["brand"] = line_data[1].split(":")[-1]
                metadata["model"] = line_data[2].split(":")[-1]
                continue
                
            if "type:" in line_data[1] :
                if "Accelerometer" in line_data[2] and "Uncalibrated" not in line_data[2] :
                    metadata["accelerometer_name"] = line_data[2].split(":")[-1].split(" ")[0]
                
                if "Gyroscope" in line_data[2] and "Uncalibrated" not in line_data[2] :
                    metadata["gyroscope_name"] = line_data[2].split(":")[-1].split(" ")[0]
                
                if "Magnetometer" in line_data[2] and "Uncalibrated" not in line_data[2] :
                    metadata["magnetometer_name"] = line_data[2].split(":")[-1].split(" ")[0]
                
                continue
            continue

        line_data = line_data.split('\t')
        
        if line_data[1] == 'TYPE_ACCELEROMETER':
            data_table.append([int(line_data[0]), # Timestamp
                               float(line_data[2]), float(line_data[3]), float(line_data[4]), # Accelerometer 
                               None, None, None, # Accelerometer uncalibrated
                               None, None, None, # Gyro
                               None, None, None, # Gyro uncalibrated
                               None, None, None, # Magnetometer
                               None, None, None, # Magnetometer uncalibrated
                               None, None, None, # AHRS
                               None, None, None, None, # WiFi
                               None, None, None, None, None, # iBeacon
                               None, None # Waypoint
                              ])
            continue

        if line_data[1] == 'TYPE_ACCELEROMETER_UNCALIBRATED':
            data_table.append([int(line_data[0]), # Timestamp
                               None, None, None, # Accelerometer 
                               float(line_data[2]), float(line_data[3]), float(line_data[4]),# Accelerometer uncalibrated
                               None, None, None, # Gyro
                               None, None, None, # Gyro uncalibrated
                               None, None, None, # Magnetometer
                               None, None, None, # Magnetometer uncalibrated
                               None, None, None, # AHRS
                               None, None, None, None, # WiFi
                               None, None, None, None, None, # iBeacon
                               None, None # Waypoint
                              ])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE':
            data_table.append([int(line_data[0]), # Timestamp
                               None, None, None, # Accelerometer 
                               None, None, None, # Accelerometer uncalibrated
                               float(line_data[2]), float(line_data[3]), float(line_data[4]), # Gyro
                               None, None, None, # Gyro uncalibrated
                               None, None, None, # Magnetometer
                               None, None, None, # Magnetometer uncalibrated
                               None, None, None, # AHRS
                               None, None, None, None, # WiFi
                               None, None, None, None, None, # iBeacon
                               None, None # Waypoint
                              ])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE_UNCALIBRATED':
            data_table.append([int(line_data[0]), # Timestamp
                               None, None, None, # Accelerometer 
                               None, None, None, # Accelerometer uncalibrated
                               None, None, None, # Gyro
                               float(line_data[2]), float(line_data[3]), float(line_data[4]), # Gyro uncalibrated
                               None, None, None, # Magnetometer
                               None, None, None, # Magnetometer uncalibrated
                               None, None, None, # AHRS
                               None, None, None, None, # WiFi
                               None, None, None, None, None, # iBeacon
                               None, None # Waypoint
                              ])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD':
            data_table.append([int(line_data[0]), # Timestamp
                               None, None, None, # Accelerometer 
                               None, None, None, # Accelerometer uncalibrated
                               None, None, None, # Gyro
                               None, None, None, # Gyro uncalibrated
                               float(line_data[2]), float(line_data[3]), float(line_data[4]), # Magnetometer
                               None, None, None, # Magnetometer uncalibrated
                               None, None, None, # AHRS
                               None, None, None, None, # WiFi
                               None, None, None, None, None, # iBeacon
                               None, None # Waypoint
                              ])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD_UNCALIBRATED':
            data_table.append([int(line_data[0]), # Timestamp
                               None, None, None, # Accelerometer 
                               None, None, None, # Accelerometer uncalibrated
                               None, None, None, # Gyro
                               None, None, None, # Gyro uncalibrated
                               None, None, None, # Magnetometer
                               float(line_data[2]), float(line_data[3]), float(line_data[4]), # Magnetometer uncalibrated
                               None, None, None, # AHRS
                               None, None, None, None, # WiFi
                               None, None, None, None, None, # iBeacon
                               None, None # Waypoint
                              ])
            continue

        if line_data[1] == 'TYPE_ROTATION_VECTOR':
            data_table.append([int(line_data[0]), # Timestamp
                               None, None, None, # Accelerometer 
                               None, None, None, # Accelerometer uncalibrated
                               None, None, None, # Gyro
                               None, None, None, # Gyro uncalibrated
                               None, None, None, # Magnetometer
                               None, None, None, # Magnetometer uncalibrated
                               float(line_data[2]), float(line_data[3]), float(line_data[4]), # AHRS
                               None, None, None, None, # WiFi
                               None, None, None, None, None, # iBeacon
                               None, None # Waypoint
                              ])
            continue

        if line_data[1] == 'TYPE_WIFI':
            data_table.append([int(line_data[0]), # Timestamp
                               None, None, None, # Accelerometer 
                               None, None, None, # Accelerometer uncalibrated
                               None, None, None, # Gyro
                               None, None, None, # Gyro uncalibrated
                               None, None, None, # Magnetometer
                               None, None, None, # Magnetometer uncalibrated
                               None, None, None, # AHRS
                               line_data[2], line_data[3], float(line_data[4]), line_data[6], # WiFi
                               None, None, None, None, None, # iBeacon
                               None, None # Waypoint
                              ])
            continue

        if line_data[1] == 'TYPE_BEACON':
            data_table.append([int(line_data[0]), # Timestamp
                               None, None, None, # Accelerometer 
                               None, None, None, # Accelerometer uncalibrated
                               None, None, None, # Gyro
                               None, None, None, # Gyro uncalibrated
                               None, None, None, # Magnetometer
                               None, None, None, # Magnetometer uncalibrated
                               None, None, None, # AHRS
                               None, None, None, None, # WiFi
                               line_data[2], line_data[3], line_data[4], '_'.join([line_data[2], line_data[3], line_data[4]]), float(line_data[6]), # iBeacon
                               None, None # Waypoint
                              ])
            continue

        if line_data[1] == 'TYPE_WAYPOINT':
            data_table.append([int(line_data[0]), # Timestamp
                               None, None, None, # Accelerometer 
                               None, None, None, # Accelerometer uncalibrated
                               None, None, None, # Gyro
                               None, None, None, # Gyro uncalibrated
                               None, None, None, # Magnetometer
                               None, None, None, # Magnetometer uncalibrated
                               None, None, None, # AHRS
                               None, None, None, None, # WiFi
                               None, None, None, None, None, # iBeacon
                               float(line_data[2]), float(line_data[3]) # Waypoint
                              ])
            continue

    
    df = pd.DataFrame(data_table)
    df.columns = ["timestamp", 
                  "acc_x", "acc_y", "acc_z", 
                  "acc_uncali_x", "acc_uncali_y", "acc_uncali_z",
                  "gyro_x", "gyro_y", "gyro_z",
                  "gyro_uncali_x", "gyro_uncali_y", "gyro_uncali_z",
                  "magn_x", "magn_y", "magn_z",
                  "magn_uncali_x", "magn_uncali_y", "magn_uncali_z",
                  "rotvec_x", "rotvec_y", "rotvec_z",
                  "wifi_ssid", "wifi_bssid", "wifi_rssi", "wifi_lastseen_ts",
                  "ib_uuid", "ib_major", "ib_minor", "ib_uuid_major_minor", "ib_rssi",
                  "waypoint_x", "waypoint_y"
                 ]
    
    if return_metadata:
        return df, metadata
    else:
        return df
    
    
def read_data_file(filepath):
    filepath_sp = filepath.split("/")
    floor = filepath_sp[-2]
    filename = filepath_sp[-1]

    train_only_metadata_keys = ["brand", "model", # only for train
                                "accelerometer_name",# only for train
                                "gyroscope_name",# only for train
                                "magnetometer_name"]# only for train
    
    df, metadata = read_one_data_file(filepath)
    df["site_id"] = metadata["site_id"] 
    df["floor"] = floorname2num(floor) # if test file, this returns None
    for key in train_only_metadata_keys:
        if key in metadata.keys():
            df[key] = metadata[key]
        else:
            df[key] = None
    df["filename"] = filename
    
    return df

## Lets read a data for example.

Train data 

In [ ]:
# manualy set file names for now.
filepath = "../input/indoor-location-navigation/train/5a0546857ecc773753327266/B1/5e15730aa280850006f3d005.txt"

df = read_data_file(filepath)
df

Test data

In [ ]:
# manualy set file names for now.
filepath = "../input/indoor-location-navigation/test/01c41f1aeba5c48c2c4dd568.txt"

df = read_data_file(filepath)
df